In [1]:
%load_ext autoreload
%autoreload 2
%pdb

# Load Graph and Landscape

In [1]:
import os
import matplotlib.pyplot as plt
dirs = ['../hari'] 
dir_ = dirs[0]
graph_folders = [folder for folder in os.listdir(dir_)]

In [2]:
def max_landscape(data):
    maximum = 0
    max_key = None
    for k, _ in data.items():
        if 'landscape' in k:
            max_beta = float(k.split('_')[2][1:])
            max_gamma = float(k.split('_')[3][1:])
            min_beta = float(k.split('_')[4][1:])
            min_gamma = float(k.split('_')[5][1:])
            disc = int(k.split('_')[1][1:])
            return data.get(k), max_beta, min_beta, max_gamma, min_gamma

In [3]:
from networkx.algorithms.isomorphism import is_isomorphic
def isomorphism_classes(graphs):
    classes = []
    for g in graphs:
        graph = g[0]
        appended = False
        for class_ in classes:
            if is_isomorphic(graph, class_[0][0]):
                class_.append(g)
                appended = True
                break
        if not appended:
            classes.append([g])
    return classes

def prune_graphs(classes):
    for class_ in classes:
        while len(class_) > 1:
            duplicate = class_.pop()
            os.remove(duplicate[1])

In [4]:
from classical_optimization.terra.utils import read_graph
from classical_optimization.qaoa_circuits import plot_landscape
def clean_up_graphs():
    for folder in graph_folders:
        path = os.path.join(dir_, folder)
        files = [f for f in os.listdir(path)]
        graphs = []
        for f in files:
            if 'pkl' in f:
                f = os.path.join(dir_, folder, f)
                data = read_graph(f)
                graphs.append((data['graph'], f))
                #DELETES FILES, UNCOMMENT CAREFULLY
                if len(data) == 1:
                    os.remove(f)
        prune_graphs(isomorphism_classes(graphs))

In [5]:
import networkx as nx
graph_sizes = []
landscapes = []
for folder in graph_folders:
    these_graphs = []
    these_landscapes = []
    path = os.path.join(dir_, folder)
    files = [f for f in os.listdir(path)]
    graphs = []
    for f in files:
        f = os.path.join(dir_, folder, f)
        graph_data = read_graph(f)
        plot_data = max_landscape(graph_data)
        if plot_data is not None:
            data, max_beta, min_beta, max_gamma, min_gamma = plot_data
            these_landscapes.append(data)
            these_graphs.append(graph_data['graph'])
            #nx.draw(graph_data['graph'])
            #plt.show()
            #plot_landscape(data, max_gamma, max_beta)
            #plt.show()
    graph_sizes.append(these_graphs)
    landscapes.append(these_landscapes)

# Utils

In [6]:
from scipy.optimize import dual_annealing
from classical_optimization.qaoa_circuits import execute_qaoa_circuit_and_estimate_cost
import numpy as np
from qiskit import Aer, execute
from coldquanta.qiskit_tools.modeling.neutral_atom_noise_model import create_noise_model

np.random.seed(666)
reprate = 50 
one_hour = 60 * 60 #seconds
max_gamma = 2 * np.pi
max_beta = np.pi
simulator = Aer.get_backend('qasm_simulator')
noise_model = create_noise_model(cz_fidelity=1)

def weights(graph):
    rtn = {}
    for e in graph.edges:
        weight = graph.get_edge_data(e[0], e[1])['weight']
        rtn[e] = weight
    return rtn

def objective(graph):
    #Hack for backwards compatibility.
    num_rows = len(graph.nodes)
    num_cols = 1

    history = []
    def store_log(func):
        def logged_func(x):
            history.append(x)
            return func(x)
        return logged_func

    @store_log
    def gamma_beta_objective(gamma_beta):
        # The cut value is the expectation value, minima of the negation correspond to maxima.
        return execute_qaoa_circuit_and_estimate_cost(gamma=gamma_beta[1], beta=gamma_beta[0],
                                                       num_shots=shots_per_point,
                                                       simulator=simulator,
                                                       coupling_map=None,
                                                       weights=weights(graph),
                                                       rows=num_rows,
                                                       cols=num_cols,
                                                       noise_model=noise_model,
                                                       # Just a fully random seed, in the full range.
                                                       seed=np.random.randint(0,2**32 - 1))
    return gamma_beta_objective, history



# An annealing attempt on a graph.

In [8]:
from tqdm import tqdm
annealing_attempts = []
for size in tqdm(graph_sizes[:1]):
    size_attempts = []
    for graph in size[:1]:
        func, history = objective(graph)
        initial_gamma_beta = [np.random.rand() * max_param for max_param in (max_gamma, max_beta)]
        result = dual_annealing(
            lambda x: -1*func(x),
            bounds=[(0, max_gamma),
                    (0, max_beta)],
            x0=np.array(initial_gamma_beta),
            # One annealing attempt.
            maxiter=1,
            initial_temp=10,
            maxfun=one_hour*reprate,
            restart_temp_ratio=1E-10,
            no_local_search=True)
        result.fun = -result.fun
        size_attempts.append((result.x, result.fun))
    annealing_attempts.append(size_attempts)
        

100%|██████████| 1/1 [00:00<00:00, 16.16it/s]


In [9]:
annealing_attempts

[[(array([3.42032777, 2.65209056]), 2.221851097028419)]]

# An ES attempt on a graph

In [14]:
from es import SimpleGA, CMAES, PEPG, OpenES
# defines OpenAI's ES algorithm solver. Note that we needed to anneal the sigma parameter
NPARAMS = 2
NPOPULATION = 100
oes = OpenES(NPARAMS,                  # number of model parameters
            sigma_init=0.025,            # initial standard deviation
            sigma_decay=0.999,         # don't anneal standard deviation
            learning_rate=0.005,         # learning rate for standard deviation
            learning_rate_decay = 0.0, # annealing the learning rate
            popsize=NPOPULATION,       # population size
            antithetic=False,          # whether to use antithetic sampling
            weight_decay=0.00,         # weight decay coefficient
            rank_fitness=False,        # use rank rather than fitness numbers
            forget_best=False)

In [15]:
MAX_ITERATION = 500
fit_func, history = objective(graph)
# defines a function to use solver to solve fit_func
def test_solver(solver):
    history = []
    for j in range(MAX_ITERATION):
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)
        for i in range(solver.popsize):
            fitness_list[i] = fit_func(solutions[i])
        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        if (j+1) % 100 == 0:
            print("fitness at iteration", (j+1), result[1])
    print("local optimum discovered by solver:\n", result[0])
    print("fitness score at this local optimum:", result[1])
    return history, result

In [16]:
es_attempts = []
for size in tqdm(graph_sizes[:1]):
    size_attempts = []
    for graph in size[:1]:
        history, result = test_solver(oes)
        size_attempts.append((result[0], result[1]))
    es_attempts.append(size_attempts)

  0%|          | 0/1 [00:00<?, ?it/s]

fitness at iteration 100 2.7498437683995642
fitness at iteration 200 2.7498437683995642
fitness at iteration 300 2.7498437683995642
fitness at iteration 400 2.7530818991162436


100%|██████████| 1/1 [09:29<00:00, 569.19s/it]

fitness at iteration 500 2.764788877515453
local optimum discovered by solver:
 [-0.00966231  0.00114483]
fitness score at this local optimum: 2.764788877515453


# Compute Maxcut of Graphs

In [25]:
def cutsize(set1, set2, g):
    cut = 0
    for s1 in set1:
        for s2 in set2:
            if g.get_edge_data(s1, s2) is not None:
                cut += g.get_edge_data(s1, s2)['weight']
    return cut

def maxcut(g, a=[], b=[], used=[]):
    for node in g.nodes:
        if node not in used:
            left = maxcut(g, list(a) + [node], list(b), list(used) + [node])[0]
            right = maxcut(g, list(a), list(b) + [node], list(used) + [node])[0]
            if left > right:
                a = list(a) + [node]
                b = list(b)
            else:
                a = list(a)
                b = list(b) + [node]
    # There are no unused nodes, we've reached a leaf.
    return cutsize(a, b, g), a, b

maxcuts = []
for size in tqdm(graph_sizes[:1]):
    size_landscapes = []
    for graph in size[:1]:
        size_landscapes.append(maxcut(graph))
    maxcuts.append(size_landscapes)

100%|██████████| 1/1 [00:00<00:00, 163.22it/s]


In [26]:
maxcuts

[[(2.860063770067472, [1, 3], [0, 2])]]

# Compute Maximum Landscape Value for Concentration

In [31]:
from classical_optimization.qaoa_circuits import plot_landscape
import matplotlib.pyplot as plt
maxargs = []
for size in tqdm(landscapes[:1]):
    size_args = []
    for landscape in size[:1]:
        size_args.append((np.argmax(landscape), np.max(landscape)))
    maxargs.append(size_args)

100%|██████████| 1/1 [00:00<00:00, 1633.93it/s]
